In [ ]:
# Set the working directory, change to your specific directory
import os
os.chdir('/Users/charleslego/my_documents/ETH/Classes/Sem3/Deep_learning/Project/Code_Project/DLProject-MLP/src')

import os
import torch
from PIL import Image
import torchvision.transforms as transforms
import sys
import matplotlib.pyplot as plt
import torch
from pathlib import Path
import numpy as np

classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

act_max_path = 'Act_Max_Img_MLP'
env_path = '.Act_Max_Img_MLP'
if env_path not in sys.path:
    sys.path.append(str(env_path))

shape0 = [1,1,1,1,3,3,3]
shape1 = [96,128,64,192,64,32,128]
shape2 = [128,96,192,64,64,128,32]
# The factors of 12288 are The factors of 12288 are: [1, 2, 3, 4, 6, 8, 12, 16, 24, 32, 48, 64, 96, 128, 192, 256, 384, 512, 768, 1024, 1536, 2048, 3072, 4096, 6144, 12288]

def norm_01(array):
    return(array-np.min(array))/(np.max(array)-np.min(array))

def save_im(batch, filepath, ch, ht, wd):
    if ch==1:
      img = batch.reshape(ht, wd)
      img_np = img.detach().numpy()
      plt.imsave(filepath, norm_01(img_np), cmap='gray')
    else:
      img = batch.reshape(ch, ht, wd).permute(1,2,0)
      img_np = img.detach().numpy()
      plt.imsave(filepath, norm_01(img_np))

def convert_pt_to_image(input_path, output_path):
    # Ensure the output directory exists
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # List all files in the input directory
    files = os.listdir(input_path)

    # Filter files with ".pt" extension
    pt_files = [f for f in files if f.endswith(".pt")]

    #List of files that couldn't be processed
    err_files = []

    for pt_file in pt_files:

        print(f"Processing file {pt_file}")
        pt_filepath = os.path.join(input_path, pt_file)
        # print(pt_filepath)

        try:
          # Load PyTorch tensor from the file
          pt_tensor = torch.load(pt_filepath, map_location=torch.device('cpu'))

          # Find the right output directory
          class_index = int(pt_file.split("_")[1])

          if 0 <= class_index < len(classes):
            class_name = classes[class_index]

            # Change filename to .png
            image_filename = os.path.splitext(pt_file)[0] + ".png"
            # Specify subfolder based on class
            image_filefolder = os.path.join(output_path, class_name)
            if not os.path.exists(image_filefolder):
              os.makedirs(image_filefolder)

            #Loop over different shapes
            for ch, ht, wd in zip(shape0, shape1, shape2):
              # Specify subfolder based on output image shape
              subfol = f"shape_{ch}x{ht}x{wd}"
              image_fileshape = os.path.join(image_filefolder, subfol)
              if not os.path.exists(image_fileshape):
                os.makedirs(image_fileshape)

              image_filepath = os.path.join(image_fileshape, image_filename)
              save_im(pt_tensor, image_filepath, ch, ht, wd)

            print(f"Successfully saved file {pt_file} as an image")

        except Exception as e:
          err_files.append(pt_file)
          print(f"An error occurred in file {pt_file}: {e}")

    err_file_path = "./Act_Max_Img_MLP_Raw_Images/err_log.txt"
    with open(err_file_path, "w") as file:
        for item in err_files:
            file.write(f"{item}\n")


# Example usage:
input_directory = "./Act_Max_Img_MLP"  # Replace with the actual path
output_directory = "./Act_Max_Img_MLP_Raw_Images"  # Replace with the actual path

convert_pt_to_image(input_directory, output_directory)
